In [1]:
import pandas as pd
import os

In [2]:
wide_files = ['gdp.csv', 'gdp_growth.csv', 'gdp_ppp.csv']
wide_paths = [os.path.join('datasets', f) for f in wide_files]

long_files = [
    'foreign-direct-investment-net-inflows-as-share-of-gdp.csv',
    'foreign-direct-investment-net-outflows-as-share-of-gdp.csv',
    'imports-of-goods-and-services-constant-2010-us.csv',
    'population-with-un-projections.csv',
    'trade-as-share-of-gdp.csv',
    'exports-of-goods-and-services-constant-2010-us.csv'
    ]
long_paths = [os.path.join('datasets', f) for f in long_files]

In [3]:
dfs = []

metric_map = {
    'gdp.csv': 'gdp',
    'gdp_growth.csv': 'gdp_growth',
    'gdp_ppp.csv': 'gdp_ppp'
}

for path in wide_paths:
    
    filename = os.path.basename(path)
    metric_name = metric_map.get(filename, filename.replace('.csv', '').replace('_', ' ').title())
    df = pd.read_csv(path)
    
    # removing some unnamed columns without data
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    
    id_vars = ['Country Name', 'Code']
    value_vars = [c for c in df.columns if c not in id_vars]
    
    df_melted = df.melt(id_vars=id_vars, value_vars=value_vars, var_name='Year', value_name=metric_name)
    
    df_melted = df_melted.rename(columns={'Country Name': 'Entity'})
    
    df_melted['Year'] = df_melted['Year'].astype(int)
    
    dfs.append(df_melted)


for path in long_paths:
    
    filename = os.path.basename(path)
    df = pd.read_csv(path)
    
    df['Year'] = df['Year'].astype(int)

    # the population dataset included estimates starting from 2024
    if 'population-with-un-projections.csv' in filename:
        df = df[df['Year'] <= 2023]

    dfs.append(df)


final_df = dfs[0]

for df in dfs[1:]:
    final_df = pd.merge(final_df, df, on=['Entity', 'Code', 'Year'], how='outer')

final_df = final_df.sort_values(by=['Entity', 'Year'])
final_df = final_df.reset_index(drop = True)


In [4]:
# removing a redundant column that the population dataset had
final_df = final_df.drop(columns=['Population - Sex: all - Age: all - Variant: medium'])

In [5]:
# renaming columns
new_names = {
    'Entity': 'country',
    'Code': 'code',
    'Year': 'year',
    'gdp': 'gdp', # USD $
    'gdp_growth': 'gdp_growth', # percentage %
    'gdp_ppp': 'gdp_ppp', # international dollars $
    'Foreign direct investment, net inflows (% of GDP)': 'fdi_inflows', # % of GDP
    'Foreign direct investment, net outflows (% of GDP)': 'fdi_outflows', # % of GDP
    'Imports of goods and services (constant 2015 US$)': 'imports', # contstant 2015 USD $
    'Population - Sex: all - Age: all - Variant: estimates': 'population', # count
    'Trade (% of GDP)': 'trade', # % of GDP
    'Exports of goods and services (constant 2015 US$)': 'exports' # contstant 2015 USD $
}

final_df = final_df.rename(columns=new_names)

In [6]:
# reordering columns
new_order = ['country', 'code', 'year', 'gdp', 'gdp_growth', 'gdp_ppp', 'fdi_inflows', 'fdi_outflows', 'imports', 'exports', 'trade', 'population']

final_df = final_df[new_order]

In [7]:
final_df

,country,code,year,gdp,gdp_growth,gdp_ppp,fdi_inflows,fdi_outflows,imports,exports,trade,population
0,Afghanistan,AFG,1950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7776180.0
1,Afghanistan,AFG,1951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7879343.0
2,Afghanistan,AFG,1952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7987784.0
3,Afghanistan,AFG,1953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8096703.0
4,Afghanistan,AFG,1954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8207954.0
...,...,...,...,...,...,...,...,...,...,...,...,...
24404,Zimbabwe,ZWE,2020,1.805117e+10,-6.248748,5.257540e+10,0.559613,-0.013026,1.948442e+09,1.736087e+09,47.313380,15526887.0
24405,Zimbabwe,ZWE,2021,NaN,NaN,NaN,0.871791,0.009178,3.146538e+09,2.552880e+09,50.847122,15797220.0
24406,Zimbabwe,ZWE,2022,NaN,NaN,NaN,1.027034,0.177800,4.846439e+09,3.661619e+09,64.763610,16069061.0
24407,Zimbabwe,ZWE,2023,NaN,NaN,NaN,1.583455,0.086571,4.311510e+09,3.355619e+09,50.794964,16340829.0


In [9]:
final_df.isna().sum()

country             0
code             1817
year                0
gdp             11569
gdp_growth      12256
gdp_ppp         17168
fdi_inflows     14601
fdi_outflows    16673
imports         16624
exports         16623
trade           14750
population       5465
dtype: int64

In [12]:
rows_missing_code = final_df[final_df['code'].isnull()]
rows_missing_code_list = rows_missing_code['country'].unique()
rows_missing_code_list

array(['Africa (UN)', 'Americas (UN)', 'Asia (UN)',
       'East Asia and Pacific (WB)', 'Europe (UN)',
       'Europe and Central Asia (WB)', 'European Union (27)',
       'High-income countries', 'Land-locked developing countries (LLDC)',
       'Latin America and Caribbean (WB)',
       'Latin America and the Caribbean (UN)',
       'Least developed countries', 'Less developed regions',
       'Less developed regions, excluding China',
       'Less developed regions, excluding least developed countries',
       'Low-income countries', 'Lower-middle-income countries',
       'Middle East, North Africa, Afghanistan and Pakistan (WB)',
       'More developed regions', 'North America (WB)',
       'Northern America (UN)', 'Oceania (UN)',
       'Small island developing states (SIDS)', 'South Asia (WB)',
       'Sub-Saharan Africa (WB)', 'Upper-middle-income countries'],
      dtype=object)

In [ ]:
# dropping rows without code, since they contain regional data
final_df = final_df.dropna(subset=['code'])

In [ ]:
unique_counts = final_df.nunique()
unique_counts

country           315
code              289
year               75
gdp             12743
gdp_growth      12034
gdp_ppp          7187
fdi_inflows      9035
fdi_outflows     6348
imports          7262
exports          7265
trade            9003
population      17490
dtype: int64

In [ ]:
# saving our final dataset
output_file = 'globalization.csv'
final_df.to_csv(output_file, index=False)